In [145]:
# 00_setup | Kütüphane Kurulumu ve Ortam Ayarı

import sys
import subprocess

# Gerekli kütüphaneleri kontrol et ve kur
required_packages = ["pandas", "openai", "tiktoken"]
for pkg in required_packages:
    try:
        __import__(pkg)
    except ImportError:
        print(f"'{pkg}' paketi yüklü değil, kuruluyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


In [146]:
# 01_settings_check | Proje Başlangıç Ayarları ve Veri Yükleme (güçlendirilmiş versiyon)

import os
import json
import warnings
from datetime import datetime
import pandas as pd
import openai
from openai import OpenAI, AuthenticationError

# Sürüm bilgisi
print("Pandas Sürümü:", pd.__version__, "| OpenAI Sürümü:", openai.__version__)

# OpenAI istemcisi başlat
try:
    client = OpenAI()  # OPENAI_API_KEY ortamdan okunur
    print("OpenAI istemcisi başarıyla başlatıldı.")
except AuthenticationError:
    raise EnvironmentError("OPENAI_API_KEY ortam değişkeni tanımlı değil veya geçersiz. Lütfen ayarlayın.")

# Hedef model snapshot adı
TARGET_MODEL = "gpt-4o-mini-2024-07-18"
print(f"Hedef Model: {TARGET_MODEL}")

# Veri dosyasını yükle
DATA_PATH = "cleaned_result.json"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'{DATA_PATH}' dosyası bulunamadı. Lütfen dizini kontrol edin.")

try:
    with open(DATA_PATH, encoding="utf-8") as f:
        raw_data = json.load(f)
    df = pd.DataFrame(raw_data)
    print(f"'{DATA_PATH}' dosyasından {len(df)} kayıt yüklendi.")
except json.JSONDecodeError:
    raise ValueError(f"'{DATA_PATH}' geçerli bir JSON formatında değil.")
except Exception as e:
    raise RuntimeError(f"Veri yükleme sırasında beklenmedik bir hata oluştu: {e}")

# Gerekli sütunları kontrol et
required_columns = [
    "İş ID", "Lokasyon", "Sektör", "İş Gönderim Saati", "İade Talebi Saati",
    "Hizmet Veren Geri Arama Saatleri", "Hizmet Veren Arama Metinleri",
    "Partner Geri Arama Saatleri", "Partner Arama Metinleri",
    "İade Talebi Nedeni", "Partner Red Sebebi", "Müşteri Cevabı", "Sistem Cevabı",
    "İade Durumu"
]

missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise KeyError(f"Veri setinde eksik zorunlu sütunlar bulundu: {', '.join(missing_cols)}")

print("Tüm gerekli sütunlar mevcut.")

# 'İade Durumu' filtreleme
valid_statuses = ["Personel Kabul Etti", "Personel Redetti"]
df = df[df["İade Durumu"].isin(valid_statuses)].copy()

print(f"\nVeri Yükleme ve Filtreleme Özeti:")
print(f"  Geçerli kayıt sayısı: {len(df)}")

# Sınıf dağılımı analizi
status_counts = df["İade Durumu"].value_counts()
print(f"  'İade Durumu' dağılımı:\n{status_counts}")

# Dengesizlik ve tek sınıf uyarısı
if len(status_counts) <= 1:
    raise ValueError("Yalnızca tek bir sınıf bulundu. Fine-tuning için en az iki sınıf gereklidir.")

min_count = status_counts.min()
max_count = status_counts.max()
if min_count / max_count < 0.2:
    warnings.warn("UYARI: 'İade Durumu' sınıflarında ciddi dengesizlik var. Bu model performansını etkileyebilir.")

# İlk kayıtlar
print(f"\nİşlenecek DataFrame boyutu: {len(df)} kayıt")
df.head(3)


Pandas Sürümü: 2.3.1 | OpenAI Sürümü: 1.97.1
OpenAI istemcisi başarıyla başlatıldı.
Hedef Model: gpt-4o-mini-2024-07-18
'cleaned_result.json' dosyasından 3000 kayıt yüklendi.
Tüm gerekli sütunlar mevcut.

Veri Yükleme ve Filtreleme Özeti:
  Geçerli kayıt sayısı: 2531
  'İade Durumu' dağılımı:
İade Durumu
Personel Kabul Etti    2303
Personel Redetti        228
Name: count, dtype: int64

İşlenecek DataFrame boyutu: 2531 kayıt


/var/folders/ng/1b3xfg1d3gbbbqgbn43lfd880000gn/T/ipykernel_75541/3079565270.py:73: UserWarning: UYARI: 'İade Durumu' sınıflarında ciddi dengesizlik var. Bu model performansını etkileyebilir.
  warnings.warn("UYARI: 'İade Durumu' sınıflarında ciddi dengesizlik var. Bu model performansını etkileyebilir.")


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,None,None,None
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,None,None,İş gönderimi ile müşterinin aranma saati arası...
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,None,None,None


In [147]:
# 02_data_cleaning | Veri Temizleme ve Temel Özellik Mühendisliği

# Tarih sütunlarını datetime'a çevir
date_cols = ["İş Gönderim Saati", "İade Talebi Saati"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

# Metin sütunlarındaki NaN'ları boş string yap
text_cols = [
    "İade Talebi Nedeni", "Hizmet Veren Arama Metinleri", "Partner Arama Metinleri",
    "Partner Red Sebebi", "Müşteri Cevabı", "Sistem Cevabı", "Lokasyon", "Sektör"
]
for c in text_cols:
    df[c] = df[c].fillna("").astype(str)

# Geçen süreyi (dakika) hesapla ve temizle
df["Time_Elapsed_Minutes"] = (
    (df["İade Talebi Saati"] - df["İş Gönderim Saati"]).dt.total_seconds() / 60
).apply(lambda x: 0 if pd.isna(x) or x < 0 else x)

# Bilgilendirici kontroller
print("Veri temizliği ve temel özellik mühendisliği tamamlandı.")
print(f"Güncel kayıt sayısı: {len(df)}")
print(f"'Time_Elapsed_Minutes' sütununda NaN kontrolü: {df['Time_Elapsed_Minutes'].isnull().sum()}")
print(f"'Time_Elapsed_Minutes' sütununun ilk 5 değeri:\n{df['Time_Elapsed_Minutes'].head()}")

df.head(3)


Veri temizliği ve temel özellik mühendisliği tamamlandı.
Güncel kayıt sayısı: 2531
'Time_Elapsed_Minutes' sütununda NaN kontrolü: 0
'Time_Elapsed_Minutes' sütununun ilk 5 değeri:
0    32.950000
1    88.833333
2     9.616667
3     5.850000
4     7.200000
Name: Time_Elapsed_Minutes, dtype: float64


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı,Time_Elapsed_Minutes
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,,,,32.950000
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,,,İş gönderimi ile müşterinin aranma saati arası...,88.833333
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,,,,9.616667


In [148]:
# 03_feature_call_counts | Arama ve Geri Arama Sayılarını Hesapla

def count_segments(value):
    """
    '||' ile ayrılmış metin alanlarındaki segment sayısını hesaplar.
    Boş veya NaN ise 0 döner.
    """
    if pd.isna(value) or str(value).strip() == "":
        return 0
    return len(str(value).split("||"))

# Arama metinlerine göre sayısal özellikler
df["HizmetVerenAramaSayisi"] = df["Hizmet Veren Arama Metinleri"].apply(count_segments)
df["PartnerAramaSayisi"] = df["Partner Arama Metinleri"].apply(count_segments)

# Geri arama saatlerine göre sayısal özellikler
df["HizmetVerenGeriAramaSayisi"] = df["Hizmet Veren Geri Arama Saatleri"].apply(count_segments)
df["PartnerGeriAramaSayisi"] = df["Partner Geri Arama Saatleri"].apply(count_segments)

# Özet
print("Arama ve Geri Arama Sayısı Özellikleri Eklendi.\n")
print("İstatistiksel Özet:")
print(df[[
    "HizmetVerenAramaSayisi", "PartnerAramaSayisi",
    "HizmetVerenGeriAramaSayisi", "PartnerGeriAramaSayisi"
]].describe().to_string())

df.head(3)


Arama ve Geri Arama Sayısı Özellikleri Eklendi.

İstatistiksel Özet:
       HizmetVerenAramaSayisi  PartnerAramaSayisi  HizmetVerenGeriAramaSayisi  PartnerGeriAramaSayisi
count             2531.000000         2531.000000                 2531.000000             2531.000000
mean                 1.882655            0.835243                    1.890162                0.836428
std                  1.547936            1.298874                    1.547849                1.300088
min                  0.000000            0.000000                    1.000000                0.000000
25%                  1.000000            0.000000                    1.000000                0.000000
50%                  1.000000            0.000000                    1.000000                0.000000
75%                  2.000000            1.000000                    2.000000                1.000000
max                 24.000000           17.000000                   24.000000               17.000000


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı,Time_Elapsed_Minutes,HizmetVerenAramaSayisi,PartnerAramaSayisi,HizmetVerenGeriAramaSayisi,PartnerGeriAramaSayisi
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,,,,32.950000,1,0,1,0
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,,,İş gönderimi ile müşterinin aranma saati arası...,88.833333,1,0,1,0
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,,,,9.616667,1,0,1,0


In [149]:
import json, pathlib

# Boş veya NaN değerler için placeholder döndürür
def default(val, placeholder="(no data)"):
    if pd.isna(val) or str(val).strip() == "":
        return placeholder
    return str(val).strip()

# SYSTEM PROMPT
SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n"
    "- Partner Rejection Reason: The reason provided by the partner if they rejected the request.\n"
    "- Customer Response: Partner's notes from customer interaction.\n"
    "- System Response: Notes from automated rules.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

# Maksimum karakter sınırı belirle (örneğin: 200)
MAX_REASON_CHAR = 200

out = []

for _, row in df.iterrows():
    user_parts = [
        f"Reason for Refund Request: {default(row['İade Talebi Nedeni'])}",
        f"Service Provider Call Transcripts: {default(row['Hizmet Veren Arama Metinleri'])}",
        f"Service Provider Call Count: {default(row['HizmetVerenAramaSayisi'])}",
        f"Partner Call Transcripts: {default(row['Partner Arama Metinleri'])}",
        f"Partner Call Count: {default(row['PartnerAramaSayisi'])}",
        f"Time Elapsed Between Job Submission and Refund Request: {int(row['Time_Elapsed_Minutes'])} minutes.",
        f"Partner Rejection Reason: {default(row['Partner Red Sebebi'])}",
        f"Customer Response: {default(row['Müşteri Cevabı'])}",
        f"System Response: {default(row['Sistem Cevabı'])}"
    ]
    user_msg = "\n".join(user_parts)

    decision = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    reason_parts = []
    primary_reason_text = default(row['İade Talebi Nedeni'], "").lower()

    if "müşteri vazgeçti" in primary_reason_text:
        reason_parts.append("Müşteri vazgeçtiği belirtildi.")
    elif "ulaşılamıyor" in primary_reason_text:
        reason_parts.append("Müşteriye ulaşılamadığı bildirildi.")
    elif "fiyat" in primary_reason_text or "ücret" in primary_reason_text:
        reason_parts.append("Müşteri fiyatı kabul etmedi.")
    elif "parça yok" in primary_reason_text or "yedek parça" in primary_reason_text:
        reason_parts.append("Yedek parça bulunamadı.")
    else:
        reason_parts.append("İade talebi incelendi.")

    partner_red_text = default(row['Partner Red Sebebi'], "")
    if partner_red_text != "(no data)":
        reason_parts.append(f"Partner red gerekçesi: {partner_red_text}.")
    
    customer_response_text = default(row['Müşteri Cevabı'], "")
    if customer_response_text != "(no data)":
        reason_parts.append(f"Müşteri cevabı: {customer_response_text}.")
    
    system_response_text = default(row['Sistem Cevabı'], "")
    if system_response_text != "(no data)":
        reason_parts.append(f"Sistem cevabı: {system_response_text}.")

    reason_str = " ".join(reason_parts).strip()
    if not reason_str:
        reason_str = "Tüm bilgiler ışığında karar verildi."

    # Eğer sınırlamayı tamamen kaldırmak isterseniz aşağıdaki if bloğunu yorum satırı yapabilirsiniz.
    if len(reason_str) > MAX_REASON_CHAR:
        reason_str = reason_str[:MAX_REASON_CHAR - 3] + "..."

    assistant_msg = json.dumps(
        {"decision": decision, "reason": reason_str},
        ensure_ascii=False,
        separators=(",", ": ")
    )

    out.append(json.dumps(
        {"messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]},
        ensure_ascii=False
    ))

# JSONL dosyasını kaydet
JSONL_PATH = "fine_tuning_data.jsonl"
pathlib.Path(JSONL_PATH).write_text("\n".join(out), encoding="utf-8")

print(f"JSONL dosyası yazıldı: {JSONL_PATH}")
print(f"Oluşturulan toplam satır sayısı: {len(out)}")


JSONL dosyası yazıldı: fine_tuning_data.jsonl
Oluşturulan toplam satır sayısı: 2531


In [150]:
# 05_cost_estimation | Fine-Tuning Token Sayımı ve Maksimum Maliyet Hesabı

import json
import tiktoken
from pathlib import Path

# Model ve dosya ayarları
MODEL = "gpt-4o-mini-2024-07-18"
JSONL_PATH = "fine_tuning_data.jsonl"

# En güncel fiyatlar (USD / 1M token)
TRAINING_PRICE = 3.00
INPUT_PRICE = 0.30
OUTPUT_PRICE = 1.20

# Tokenizer başlat
try:
    enc = tiktoken.encoding_for_model(MODEL)
except Exception:
    raise ValueError(f"{MODEL} için geçerli bir tokenizer bulunamadı.")

# Sayaçlar
total_examples = 0
total_input_tokens = 0
total_output_tokens = 0

# Token sayımı
with open(JSONL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]

        input_text = "".join([m["content"] for m in messages if m["role"] != "assistant"])
        output_text = "".join([m["content"] for m in messages if m["role"] == "assistant"])

        total_input_tokens += len(enc.encode(input_text))
        total_output_tokens += len(enc.encode(output_text))
        total_examples += 1

total_tokens = total_input_tokens + total_output_tokens

# Maliyet hesaplama
million = lambda x: x / 1_000_000
cost_input = million(total_input_tokens) * INPUT_PRICE
cost_output = million(total_output_tokens) * OUTPUT_PRICE
cost_training = million(total_tokens) * TRAINING_PRICE
total_cost = cost_input + cost_output + cost_training

# Bilgilendirme
print("-" * 60)
print("Fine-Tuning Maliyet Tahmini (Token Bazında)")
print("-" * 60)
print(f"Model: {MODEL}")
print(f"Dosya: {JSONL_PATH}")
print(f"Toplam örnek sayısı: {total_examples}")
print()
print("Token Sayıları:")
print(f"  Input tokens:  {total_input_tokens:,}")
print(f"  Output tokens: {total_output_tokens:,}")
print(f"  Toplam tokens: {total_tokens:,}")
print()
print("Fiyatlandırma (USD / 1M token):")
print(f"  Input:    ${INPUT_PRICE}")
print(f"  Output:   ${OUTPUT_PRICE}")
print(f"  Training: ${TRAINING_PRICE}")
print()
print("Tahmini Maksimum Maliyet (USD):")
print(f"  Input:    ${cost_input:.4f}")
print(f"  Output:   ${cost_output:.4f}")
print(f"  Training: ${cost_training:.4f}")
print("-" * 40)
print(f"  Toplam:   ${total_cost:.2f}")
print()
print("Not: Bu hesaplama maksimum tahmini vermektedir.")
print("Gerçek maliyet, discarded örnekler ve OpenAI'nin tokenization farkları nedeniyle az da olsa değişebilir.")


------------------------------------------------------------
Fine-Tuning Maliyet Tahmini (Token Bazında)
------------------------------------------------------------
Model: gpt-4o-mini-2024-07-18
Dosya: fine_tuning_data.jsonl
Toplam örnek sayısı: 2531

Token Sayıları:
  Input tokens:  2,112,345
  Output tokens: 113,254
  Toplam tokens: 2,225,599

Fiyatlandırma (USD / 1M token):
  Input:    $0.3
  Output:   $1.2
  Training: $3.0

Tahmini Maksimum Maliyet (USD):
  Input:    $0.6337
  Output:   $0.1359
  Training: $6.6768
----------------------------------------
  Toplam:   $7.45

Not: Bu hesaplama maksimum tahmini vermektedir.
Gerçek maliyet, discarded örnekler ve OpenAI'nin tokenization farkları nedeniyle az da olsa değişebilir.


In [151]:
# 06_fine_tune_start | JSONL dosyasını yükle ve fine-tuning başlat

import openai
import os

# Bağımlılık kontrolü
if "client" not in globals():
    raise RuntimeError("client nesnesi tanımlı değil. Lütfen önce 01_settings_check hücresini çalıştırın.")
if "TARGET_MODEL" not in globals():
    raise RuntimeError("TARGET_MODEL tanımlı değil. 01_settings_check hücresinde belirlenmiş olmalı.")

# Dosya kontrolü
if not os.path.exists(JSONL_PATH):
    raise FileNotFoundError(f"'{JSONL_PATH}' bulunamadı. Önceki hücreyi çalıştırdığınızdan emin olun.")

try:
    # Dosyayı OpenAI'ye yükle
    with open(JSONL_PATH, "rb") as f:
        uploaded_file = client.files.create(file=f, purpose="fine-tune")
    file_id = uploaded_file.id
    print(f"Yüklendi → File ID: {file_id}")

    # Fine-tuning işlemini başlat
    job = client.fine_tuning.jobs.create(
        training_file=file_id,
        model=TARGET_MODEL,
        suffix="lipyum-v1"
    )
    job_id = job.id
    print(f"Başlatıldı → Job ID: {job_id}")
    print(f"Takip: https://platform.openai.com/fine-tuning/{job_id}")

    # Global takip için
    global FINE_TUNING_JOB_ID
    FINE_TUNING_JOB_ID = job_id

except openai.AuthenticationError:
    print("HATA: API anahtarınız eksik veya geçersiz. Ortam değişkeninizi kontrol edin.")
except openai.RateLimitError:
    print("UYARI: Rate limit sınırına ulaşıldı. Lütfen daha sonra tekrar deneyin.")
except openai.APIError as e:
    if "exceeded_quota" in str(e).lower() or "insufficient_quota" in str(e).lower():
        print("UYARI: Bakiye yetersiz veya kota aşıldı. OpenAI hesabınızı kontrol edin.")
    else:
        print(f"OpenAI API Hatası: {e}")
except Exception as e:
    print(f"Beklenmeyen Hata: {e}")


Yüklendi → File ID: file-4EbTrz2jQUUs1wZKdhJoZM
UYARI: Bakiye yetersiz veya kota aşıldı. OpenAI hesabınızı kontrol edin.


In [152]:
# 07_follow_job | Fine-tuning iş durumunu izle ve model adını kaydet

import time
from datetime import datetime

# Fine-tuning job ID kontrolü
if 'FINE_TUNING_JOB_ID' not in globals():
    print("Uyarı: FINE_TUNING_JOB_ID tanımlı değil. Lütfen önce 06_fine_tune_start hücresini çalıştırın.")
else:
    max_minutes = 60
    start_time = time.time()
    last_status = None

    while True:
        try:
            job_info = client.fine_tuning.jobs.retrieve(FINE_TUNING_JOB_ID)
            status = job_info.status

            if status != last_status:
                print(datetime.now().strftime("%H:%M:%S"), "→ Durum:", status)
                last_status = status

            if status in ("succeeded", "failed"):
                break

            if time.time() - start_time > max_minutes * 60:
                print("UYARI: 1 saat sınırı aşıldı, takip sonlandırıldı.")
                break

            time.sleep(15)

        except openai.RateLimitError:
            print("Rate limit hatası. Bekleniyor...")
            time.sleep(30)
        except openai.APIError as e:
            print(f"API hatası oluştu: {e}. Bekleniyor...")
            time.sleep(30)
        except Exception as e:
            print(f"Beklenmeyen hata oluştu: {e}. Takip devam ediyor...")
            time.sleep(30)

    if last_status == "succeeded":
        model_name = job_info.fine_tuned_model
        print("Model hazır:", model_name)

        with open("fine_tuned_model.txt", "w") as f:
            f.write(model_name)

        print("Model adı 'fine_tuned_model.txt' dosyasına kaydedildi.")

    elif last_status == "failed":
        print(f"Eğitim başarısız: {FINE_TUNING_JOB_ID}")
        error_info = getattr(job_info, "error", None)
        if error_info:
            error_msg = getattr(error_info, "message", "")
            if error_msg:
                print("Hata mesajı:", error_msg)


Uyarı: FINE_TUNING_JOB_ID tanımlı değil. Lütfen önce 06_fine_tune_start hücresini çalıştırın.


In [153]:
# 08_model_test | Fine-tuned modele örnek prompt gönder ve yanıtı al

import openai, json

# OpenAI istemcisi kontrolü
if 'client' not in globals():
    raise RuntimeError("OpenAI istemcisi tanımlı değil. Lütfen önce 01_settings_check hücresini çalıştırın.")

# Sistem mesajı: GPT’nin nasıl davranacağını tanımlar
SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n"
    "- Partner Rejection Reason: The reason provided by the partner if they rejected the request.\n"
    "- Customer Response: Partner's notes from customer interaction.\n"
    "- System Response: Notes from automated rules.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

# Fine-tuned model adını oku
try:
    with open("fine_tuned_model.txt", "r") as f:
        final_model_name = f.read().strip()
    print("Kullanılacak model:", final_model_name)
except FileNotFoundError:
    print("UYARI: 'fine_tuned_model.txt' bulunamadı.")
    final_model_name = None

# Test fonksiyonu
def evaluate(model_name: str,
            reason: str,
            provider_transcripts: str = "",
            partner_transcripts: str = "",
            provider_call_count: int = 0,
            partner_call_count: int = 0,
            elapsed_minutes: int = 0) -> dict:

    if not model_name:
        raise ValueError("Model adı tanımlı değil.")

    user_prompt = (
        f"Reason for Refund Request: {reason}\n"
        f"Service Provider Call Transcripts: {provider_transcripts}\n"
        f"Service Provider Call Count: {provider_call_count}\n"
        f"Partner Call Transcripts: {partner_transcripts}\n"
        f"Partner Call Count: {partner_call_count}\n"
        f"Time Elapsed Between Job Submission and Refund Request: {elapsed_minutes} minutes."
    )

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)

    except json.JSONDecodeError as e:
        print("Geçersiz JSON:", e)
        return {"decision": "ERROR", "reason": "Geçersiz JSON"}
    except openai.APIError as e:
        return {"decision": "API_ERROR", "reason": str(e)}
    except Exception as e:
        return {"decision": "UNKNOWN_ERROR", "reason": str(e)}

# Örnek test
if final_model_name:
    result = evaluate(
        model_name=final_model_name,
        reason="Müşteri yanlış adres vermiş, hizmet iptal edilmiş.",
        provider_transcripts="Müşteri adresi Zeytinburnu demişti ama aradığım kişi Erzurum'daydı.",
        partner_transcripts="Partner: Müşteri adres bilgisinin yanlış olduğunu doğruladı.",
        provider_call_count=2,
        partner_call_count=1,
        elapsed_minutes=22
    )
    print("\n--- Model Kararı ---")
    print("Karar   :", result.get("decision", "Belirsiz"))
    print("Gerekçe :", result.get("reason", "Belirsiz"))
else:
    print("Model adı tanımlanmadığı için test yapılmadı.")


UYARI: 'fine_tuned_model.txt' bulunamadı.
Model adı tanımlanmadığı için test yapılmadı.
